In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from gensim.models import KeyedVectors
from janome.tokenizer import Tokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
tokenizer = Tokenizer()

In [3]:
chive = KeyedVectors.load(
    "../../data/v2/chive-1.1-mc5-aunit_gensim/chive-1.1-mc5-aunit.kv"
)

In [11]:
train_df_raw = pd.read_csv("../../data/v2/train.tsv", sep="\t")
val_df_raw = pd.read_csv("../../data/v2/val.tsv", sep="\t")

In [12]:
df = pd.concat([train_df_raw, val_df_raw], axis=0)
df = df.reset_index(drop=True)
df

,text,label
0,ぼけっとしてたらこんな時間 チャリあるから食べにでたいのに,n
1,今日の月も白くて明るい 昨日より雲が少なくてキレイな と立ち止まる帰り道 チャリなし生活も悪くない,p
2,早寝するつもりが飲み物がなくなりコンビニへ ん 今日 風が涼しいな,p
3,眠い 眠れない,n
4,ただいま って新体操してるやん 外食する気満々で家に何もないのに テレビから離れられない,p
...,...,...
43195,真夜中にふと思い立ち ノートpcを持って部屋を出て ダイニングで仕事したらすんごい捗った 環...,p
43196,ぐっどこんでぃしょん 心も頭もクリア 秋分の日のおかげかな 人と自然としっとり過ごしたい気分,p
43197,朝から免許の更新へ 0分で終わり 出口へ向かうと献血の呼びかけが みんな通り過ぎていく 献血...,p
43198,夜も更けて参りましたが 食後のコーヒーが飲みたいのでドリップ開始 ぼんやり秋の夜長を楽しみた...,p


In [9]:
data_df = pd.read_csv("../../data/data_ans.txt", sep=" ", header=None)
data_df.columns = ["label", "text"]
data_df

,label,text
0,n,雨が降ってるから気分が落ち込む。早く晴れてほしいな。
1,p,友達と映画を見に行ったけど、感動して泣いちゃった。
2,p,新しい仕事が始まって緊張するけど、頑張りたい。
3,p,美味しいケーキを食べて、幸せな気持ちになった。
4,n,ペットが病気になって心配でたまらない。早く元気になって。
...,...,...
135,p,試合に勝って、チーム全員で喜んだ。
136,p,猫と一緒に過ごす時間が一番の癒し。
137,p,今日は特に何もない一日だったけど、それも悪くない。
138,p,新しいゲームを始めて、ハマってしまった。


In [4]:
def text2seq(text: str, padded_len: int) -> np.ndarray:
    tokens = list(tokenizer.tokenize(str(text)))
    assert len(tokens) <= padded_len
    seq = np.zeros(padded_len, dtype=np.int64)
    head = padded_len - len(tokens)
    seq[:head] = 0
    for i, token in enumerate(tokens):
        if token.base_form in chive:
            seq[head + i] = chive.key_to_index[token.base_form] + 2
        else:
            seq[head + i] = 1
    return seq

In [8]:
text2seq("私は猫です", 10)

array([  0,   0,   0,   0,   0,   0,  65,   9, 570,  17], dtype=int64)

In [9]:
max_len = 0
for text in tqdm(df["text"]):
    max_len = max(max_len, len(list(tokenizer.tokenize(str(text)))))
max_len

100%|██████████| 43200/43200 [00:49<00:00, 873.68it/s] 


124

In [10]:
max_len_data = 0
for text in tqdm(data_df["text"]):
    max_len_data = max(max_len_data, len(list(tokenizer.tokenize(str(text)))))
max_len_data

100%|██████████| 140/140 [00:00<00:00, 1488.28it/s]


19

In [17]:
PADDED_LEN = 128

In [11]:
X = np.stack([text2seq(text, PADDED_LEN) for text in tqdm(df["text"])])

100%|██████████| 43200/43200 [00:53<00:00, 803.71it/s] 


In [13]:
df["label"] = df["label"].map({"n": 0, "p": 1})
df

,text,label
0,ぼけっとしてたらこんな時間 チャリあるから食べにでたいのに,0
1,今日の月も白くて明るい 昨日より雲が少なくてキレイな と立ち止まる帰り道 チャリなし生活も悪くない,1
2,早寝するつもりが飲み物がなくなりコンビニへ ん 今日 風が涼しいな,1
3,眠い 眠れない,0
4,ただいま って新体操してるやん 外食する気満々で家に何もないのに テレビから離れられない,1
...,...,...
43195,真夜中にふと思い立ち ノートpcを持って部屋を出て ダイニングで仕事したらすんごい捗った 環...,1
43196,ぐっどこんでぃしょん 心も頭もクリア 秋分の日のおかげかな 人と自然としっとり過ごしたい気分,1
43197,朝から免許の更新へ 0分で終わり 出口へ向かうと献血の呼びかけが みんな通り過ぎていく 献血...,1
43198,夜も更けて参りましたが 食後のコーヒーが飲みたいのでドリップ開始 ぼんやり秋の夜長を楽しみた...,1


In [13]:
X = torch.tensor(X)
y = torch.tensor(df["label"].values, dtype=torch.int64)

In [14]:
random_state = 42
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=random_state
)

In [15]:
train_ds = TensorDataset(X_train, y_train)
val_ds = TensorDataset(X_val, y_val)

In [16]:
torch.save(train_ds, "../../data/v2/train_ds.pth")
torch.save(val_ds, "../../data/v2/val_ds.pth")

In [14]:
data_df["label"].value_counts()

label
p    105
n     35
Name: count, dtype: int64

In [15]:
data_df["label"] = data_df["label"].map({"n": 0, "p": 1})
data_df

,label,text
0,0,雨が降ってるから気分が落ち込む。早く晴れてほしいな。
1,1,友達と映画を見に行ったけど、感動して泣いちゃった。
2,1,新しい仕事が始まって緊張するけど、頑張りたい。
3,1,美味しいケーキを食べて、幸せな気持ちになった。
4,0,ペットが病気になって心配でたまらない。早く元気になって。
...,...,...
135,1,試合に勝って、チーム全員で喜んだ。
136,1,猫と一緒に過ごす時間が一番の癒し。
137,1,今日は特に何もない一日だったけど、それも悪くない。
138,1,新しいゲームを始めて、ハマってしまった。


In [18]:
X_test = np.stack([text2seq(text, PADDED_LEN) for text in tqdm(data_df["text"])])
X_test = torch.tensor(X_test)
y_test = torch.tensor(data_df["label"].values, dtype=torch.int64)

100%|██████████| 140/140 [00:00<00:00, 521.19it/s]


In [19]:
test_ds = TensorDataset(X_test, y_test)
torch.save(test_ds, "../../data/v2/test_ds.pth")